코드출처 : https://ratsgo.github.io/from%20frequency%20to%20semantics/2017/07/04/glove/

In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from scipy.spatial.distance import cosine
import glove

In [2]:
wakati_file = 'toji.wakati'
with open("./데이터//"+wakati_file, 'r', encoding='UTF-8') as fp:
    corpus = fp.readlines()

### 공기행렬(Cooccurrence Matrix) 생성코드

In [3]:
vectorizer = CountVectorizer(min_df = 10, ngram_range=(1,1))
X = vectorizer.fit_transform(corpus)
Xc = X.T * X
Xc.setdiag(0)
result = Xc.toarray()

In [4]:
dic = {}
for idx1, word1 in enumerate(result):
    tmpdic = {}
    for idx2, word2 in enumerate(word1):
        if word2 > 0:
            tmpdic[idx2] = word2
    dic[idx1] = tmpdic

In [5]:
import operator
vocab = sorted(vectorizer.vocabulary_.items(), key=operator.itemgetter(1))
vocab = [word[0] for word in vocab]

In [6]:
print(len(vocab))
print(vocab[:6])

661
['가까이', '가깝다', '가끔', '가다', '가락지', '가문']


In [7]:
#X = vectorizer.fit_transform(corpus)
print("문장갯수 :",len(corpus))
print("단어갯수 :", len(vectorizer.vocabulary_.items()))
print(corpus[:5])

문장갯수 : 3439
단어갯수 : 661
['\n', '제 1 편 어둠 발 소리\n', '서다 序\n', '1897 년 한가위\n', '까치 들 울타리 안 감나무 와 아침 인사 하다 전 무색 옷 댕기 꼬리 늘 아이 들 송편 입 물 마을 길 쏘다 기쁘다 날뛰다 어른 들 해 중천 좀 기울다 무렵 이 래야 차례 치르다 하다 성묘 하다 하다 이웃 끼리 음식 나누다 보다 한나절 넘다 이 때 타작 마당 사람 들 모이다 시작 하다 들다 뜨다 시작 하다 남정 노인 들 아낙 들 채비 아무래도 더 디어 데 그렇다 수 없다 것 식구 들 시중 음식 간수 끝내다 제 자신 치장 남아 있다 이 바람 고개 무겁다 벼 이삭 황금 빛 물결 이루다 들판 마음 놓다 새떼 들 모여들다 풍 성 향연 벌이다\n']


In [8]:
#X = vectorizer.fit_transform(corpus)
vectorizer.fit_transform(corpus)

<3439x661 sparse matrix of type '<class 'numpy.int64'>'
	with 21322 stored elements in Compressed Sparse Row format>

In [9]:
#Xc = X.T * X
vectorizer.fit_transform(corpus).T * vectorizer.fit_transform(corpus)

<661x661 sparse matrix of type '<class 'numpy.int64'>'
	with 156359 stored elements in Compressed Sparse Column format>

In [10]:
"""
Xc.setdiag(0)
"""
import pandas as pd
pd.DataFrame(Xc.toarray(),index=vocab,columns=vocab).iloc[:5,:10]

,가까이,가깝다,가끔,가다,가락지,가문,가배,가보다,가슴,가시
가까이,0,0,0,10,0,0,0,0,0,0
가깝다,0,0,0,2,0,0,0,0,0,0
가끔,0,0,0,3,0,0,0,0,0,0
가다,10,2,3,0,2,2,4,9,3,5
가락지,0,0,0,2,0,0,0,0,0,0


### 학습시키기

In [11]:
import glove
model = glove.Glove(dic, d=100, alpha=0.75, x_max=100.0)
for epoch in range(25):
    err = model.train(batch_size=200, workers=4)
    if not epoch % 5:
        print("epoch %d, error %.3f" % (epoch, err), flush=True)

epoch 0, error 0.026
epoch 5, error 0.013
epoch 10, error 0.011
epoch 15, error 0.010
epoch 20, error 0.009


In [12]:
def most_similar(word, vocab, vecs, topn=10):
    query = vecs[vocab.index(word)]
    result = []
    for idx, vec in enumerate(vecs):
        if idx is not vocab.index(word):
            result.append((vocab[idx],1-cosine(query,vec)))
    result = sorted(result,key=lambda x: x[1],reverse=True)
    return result[:topn]

In [13]:
word = '가까이'
most_similar(word=word, vocab=vocab, vecs=model.W, topn=5)

[('흔들다', 0.9549152167279664),
 ('울리다', 0.953473275166897),
 ('마루', 0.9468285727390756),
 ('들리다', 0.9370325760794095),
 ('구석', 0.9342243827472044)]

In [20]:
len(model.W[0])

100